Script de preparación para el primer conjunto de datos, que traduce la información de los sensores y actividades en vectores de característica, y extrae para cada momento, la información de los 30, 5 y último minuto.

Puesto que este proceso puede tardar más de una hora, se proporciona el fichero resultante con el nombre de data1_window_next.

Este notebook se puede ejecutar en local, aunque se recomienda su ejecución en Google Colab con GPU



In [0]:
#Para cargar los datos en google colab
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import io
import datetime as dt

In [0]:
#Se configura el directorio de Google Drive donde se haya guardado el fichero origen y donde se guardará el de destino
FILE='/content/gdrive/My Drive/data1'
WINDOW_FILE='/content/gdrive/My Drive/data1_window_next'

## Carga de datos

In [0]:

# Se cargan los datos
names=['hour', 'sensor', 'state', 'activity']
df= pd.read_csv(FILE, delimiter='\t', names=names) 

# La columna activity se separa entre la propia actividad, y el flag de inicio/fin
new = df["activity"].str.split(" ", n = 1, expand = True) 
df["activity"]= new[0] 
df["activity_flag"]= new[1] 

In [0]:
#Se sustituyen las palabras ON, OFF, OPEN, CLOSE, begin y end, por sus correspondientes valores de flag 0/1
df['state']=df.state.replace({"ON": 1, "OFF": 0}).replace({"OPEN": 1, "CLOSE": 0})
df['state']=pd.to_numeric(df.state)
df['activity_flag']=df.activity_flag.replace({"begin": 1, "end": 0})
df['activity_flag']=pd.to_numeric(df.activity_flag)
df['hour'] = pd.to_datetime(df['hour'])

In [0]:
#Se muestra la duración media de las actividades. Puede hacerse filtrando por actividad si se quita el comentario de la siguiente linea
df_act=df#.loc[df['activity']=='Master_Bathroom'].reset_index()

dif=df_act['hour'].iloc[0]-df_act['hour'].iloc[0]
for i in (range(0,df_act.shape[0])):
  if i%2!=0:
    dif=dif+df_act['hour'].iloc[i]-df_act['hour'].iloc[i-1]

dif/(df_act.shape[0]/2)

## Se transforma la columna Sensor a un embedding que se mantendrá activo todo el siempo que lo estuvo el sensor

In [0]:
#Se convierten las columas sensor-state en múltiples columnas, una para cada tipo de sensor que indicará si está activado
#Se elimina la jerarquía de múlticolumna generada en el pivot
#Se resetea el index
df_sensor=pd.pivot_table(df,index=['hour'],columns=['sensor'],values=['state'],aggfunc=np.mean,fill_value=-1)
df_sensor.columns = df_sensor.columns.get_level_values(1)
df_sensor.reset_index(level=0, inplace=True)


## Se obtiene la siguiente actividad, sin tener en cuenta la inactividad

In [0]:
#Nos quedamos con el momento inicio de las actividades
df_act=df[df['activity_flag']==1].reset_index()

#Guardamos la primera actividad, que se va a perder en el paso posterior
first_act=df_act['activity'].iloc[0]

#Subimos todas las actividades un momento, puesto que se quiere sacar la actividad futura
next_activity=df_act['activity'].drop(df_act.index[0])
next_activity=next_activity.append(pd.Series([''])).reset_index()[0]
df_act['next_activity']=next_activity

In [0]:
#Se junta la información de los sensores y la de las actividades
df_join=pd.merge(df_sensor, df_act[['hour','next_activity','activity']], on='hour', how='left').fillna(value=-1)

In [0]:
df_join.head()

,hour,D001,D002,D003,M001,M002,M003,M004,M005,M006,M007,M008,M009,M010,M011,M012,M013,M014,M015,M016,M017,M018,M019,M020,M021,M022,M023,M024,M025,M026,M027,M028,T001,T002,next_activity,activity
0,2009-10-16 00:01:04.000059,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1,-1
1,2009-10-16 00:01:06.000046,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1,-1
2,2009-10-16 00:01:07.000064,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1,-1
3,2009-10-16 00:01:08.000081,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1,-1
4,2009-10-16 00:01:09.000028,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1,-1


## Se extienden los valores 1 de los flags, desde el momento que se activa el sensor hasta que se apaga

In [0]:
#Función que extiende la activación de un flag/actividad durante todo el tiempo que está activo

def extend_columns(col):
  s1 = []
  value=-1
  for x in col:
    if (x!=-1):
      value=x
    s1.append(value)
  return pd.Series(s1) 

In [0]:
#Se extiende todos los valores de todas las columnas 
for i in range(0,df_join.shape[1]):
  if(df_join.columns[i]!='hour'):
    df_join[df_join.columns[i]] = extend_columns(df_join[df_join.columns[i]])

df_join=df_join.replace(-1,0)
#Se recupera la primera actividad que se había perdido, y se asigna a los primeros registros que han quedado sin actividad.

df_join=df_join.replace({'next_activity': {0: first_act}})

#Se eliminan los registros finales que ya no tienen ninguna actividad
df_join=df_join.loc[(df_join['next_activity']!='')]

In [0]:
#El dataframe resultante es un array en el que los flags de cada sensor, están marcados durante todo el tiempo que se mantuvo activado el sensor
#En la columna next_activity, se indica la siguiente actividad que se realizará
#La columna activity indica la actividad actual, y sí contempla los momentos de inactividad
df_join.head()

,hour,D001,D002,D003,M001,M002,M003,M004,M005,M006,M007,M008,M009,M010,M011,M012,M013,M014,M015,M016,M017,M018,M019,M020,M021,M022,M023,M024,M025,M026,M027,M028,T001,T002,next_activity,activity
0,2009-10-16 00:01:04.000059,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,Bed_to_Toilet,0
1,2009-10-16 00:01:06.000046,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,Bed_to_Toilet,0
2,2009-10-16 00:01:07.000064,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,Bed_to_Toilet,0
3,2009-10-16 00:01:08.000081,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,Bed_to_Toilet,0
4,2009-10-16 00:01:09.000028,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,Bed_to_Toilet,0


## Extracción de los datos por ventanas de 1, 5 y 30 minutos
El resultado se guarda en un fichero

In [0]:
#Se ordenan los datos según la ventana de 1, 5 y 30 minutos.
#Para cada momento, se extraen los sensores que estuvieron activados en esas ventanas de tiempo, y la actividad realizada en el momento


s0 = []
#Sensores tipo flag
sensors=['D001','D002','D003','M001','M002','M003','M004','M005','M006','M007','M008','M009','M010','M011','M012','M013',
         'M014','M015','M016','M017','M018','M019','M020','M021','M022','M023','M024','M025','M026','M027','M028']
#Sensores de temperatura
temp=['T001','T002']
df_toSave=df_join
df_toSave[temp]=df_toSave[temp].replace(0,20.5)

#Se recorren todas las posiciones del dataframe
for i in (range(0,df_toSave.shape[0])):
  #Se extrae la hora
  now=df_toSave['hour'].iloc[i]
  #Se extrae la actividad
  activity=df_toSave['next_activity'].iloc[i]

  #Se restan 1, 5 y 30 minutos al momento actual
  minute1=now-dt.timedelta(minutes=1)
  minute5=now-dt.timedelta(minutes=5)
  minute30=now-dt.timedelta(minutes=30)

  #Se extraen los datos de los últimos 1, 5 y 30 minutos
  df_1min=df_toSave.loc[(df_toSave['hour']<=now) & (df_toSave['hour']>=minute1)]
  df_5min=df_toSave.loc[(df_toSave['hour']<=now) & (df_toSave['hour']>=minute5)]
  df_30min=df_toSave.loc[(df_toSave['hour']<=now) & (df_toSave['hour']>=minute30)]

  #En cada ventana, se suman los valores de cada columna de los sensores de tipo flag. 
  #De los de tipo temperatura, se extrae la media
  vec_1min=df_1min[sensors].sum()
  vec_1min_temp=df_1min[temp].mean()
  vec_5min=df_5min[sensors].sum()
  vec_5min_temp=df_5min[temp].mean()
  vec_30min=df_30min[sensors].sum()
  vec_30min_temp=df_30min[temp].mean()
  
  #Se junta toda la información de las 3 ventanas, la del momento actual y la actividad, y se guardan en un fichero

  a=vec_1min.append(vec_1min_temp).append(vec_5min).append(vec_5min_temp).append(vec_30min).append(vec_30min_temp).append(pd.Series([activity,now]))
  s0.append(a)
  if(i%10000==0): print (i)


array=np.asarray(s0)

columns=['W1D001','W1D002','W1D003','W1M001','W1M002','W1M003','W1M004','W1M005','W1M006','W1M007','W1M008','W1M009','W1M010','W1M011','W1M012','W1M013','W1M014',
         'W1M015','W1M016','W1M017','W1M018','W1M019','W1M020','W1M021','W1M022','W1M023','W1M024','W1M025','W1M026','W1M027','W1M028','W1T001','W1T002',
         'W5D001','W5D002','W5D003','W5M001','W5M002','W5M003','W5M004','W5M005','W5M006','W5M007','W5M008','W5M009','W5M010','W5M011','W5M012','W5M013','W5M014',
         'W5M015','W5M016','W5M017','W5M018','W5M019','W5M020','W5M021','W5M022','W5M023','W5M024','W5M025','W5M026','W5M027','W5M028','W5T001','W5T002',
         'W30D001','W30D002','W30D003','W30M001','W30M002','W30M003','W30M004','W30M005','W30M006','W30M007','W30M008','W30M009','W30M010','W30M011','W30M012','W30M013',
         'W30M014','W30M015','W30M016','W30M017','W30M018','W30M019','W30M020','W30M021','W30M022','W30M023','W30M024','W30M025','W30M026','W30M027','W30M028','W30T001','W30T002','next_activity','now']



windows_df=pd.DataFrame(array,columns=columns)
windows_df.to_csv(WINDOW_FILE,sep='|')

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


0
